In [1]:
from PIL import Image
import os
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import transforms, datasets
import os
from torch.utils.data import DataLoader, Dataset
import math
from torchvision.transforms import ColorJitter, Normalize
from torch.utils.data import ConcatDataset
from torch.utils.data import Subset


In [2]:
def downsample_image(image_path, output_size):
    # Open the image
    image = Image.open(image_path)

    # Perform downsampling using bicubic interpolation
    downscaled_image = image.resize(output_size, resample=Image.BICUBIC)

    return downscaled_image

# Folder path containing the images
folder_path = "/kaggle/input/animal-faces/afhq/train/dog"

# Defining folder for downscaled images serving for input for modelling (&upscaling)
output_folder_path = "/kaggle/working/dog_low_res"
if not os.path.exists(output_folder_path):
    os.makedirs(output_folder_path)


# Output size for downsampling (by a factor of 3)
output_size = (128, 128)

# Iterate over the files in the folder
for number, filename in enumerate(os.listdir(folder_path)):

  # Construct the full path to the image file
  image_path = os.path.join(folder_path, filename)

  # Apply downsampling to the image
  downsampled_image = downsample_image(image_path, output_size)

  # Save the downscaled image
  output_filename = f"downsampled_{filename}"
  output_path = os.path.join(output_folder_path, output_filename)
  downsampled_image.save(output_path)


FileNotFoundError: [WinError 3] Das System kann den angegebenen Pfad nicht finden: '/kaggle/input/animal-faces/afhq/train/dog'

In [3]:
def downsample_image(image_path, output_size):
    # Open the image
    image = Image.open(image_path)

    # Perform downsampling using bicubic interpolation
    downscaled_image = image.resize(output_size, resample=Image.BICUBIC)

    return downscaled_image

# Folder path containing the images
folder_path = "./data/data_final"

# Defining folder for downscaled images serving for input for modelling (&upscaling)
output_folder_path = "./data/mensch_low_res"

test_low_res_folder = "./data/mensch_test_low_res"
test_original = "./data/mensch_test_original"

if not os.path.exists(output_folder_path):
    os.makedirs(output_folder_path)
if not os.path.exists(test_low_res_folder):
    os.makedirs(test_low_res_folder)
if not os.path.exists(test_original):
    os.makedirs(test_original)


# Output size for downsampling (by a factor of 3)
output_size = (128, 128)

number_images = len(os.listdir(folder_path))

# Iterate over the files in the folder
for number, filename in enumerate(os.listdir(folder_path)):

    # Check if the file is an image (optional)
    if number<=0.8*number_images:
      # Construct the full path to the image file
      image_path = os.path.join(folder_path, filename)

      # Apply downsampling to the image
      downsampled_image = downsample_image(image_path, output_size)

      # Save the downscaled image
      output_filename = f"downsampled_{filename}"
      output_path = os.path.join(output_folder_path, output_filename)
      downsampled_image.save(output_path)

    else:
      # Construct the full path to the image file
      image_path = os.path.join(folder_path, filename)

      image = Image.open(image_path)
      image.save(os.path.join(test_original, filename))


      # Apply downsampling to the image
      downsampled_image = downsample_image(image_path, output_size)

      # Save the downscaled image
      output_filename = f"downsampled_{filename}"
      output_path = os.path.join(test_low_res_folder, output_filename)
      downsampled_image.save(output_path)

      os.remove(os.path.join(folder_path, filename))


### data loading (with augmentation)

In [4]:
# low_res_folder = "./drive/MyDrive/dog_low_res"
# high_res_folder = "./drive/MyDrive/dog"


# # === creating dataset with all images ===
# class CustomDataset(Dataset):
#     def __init__(self, low_res_folder, high_res_folder, transform=None):
#         self.low_res_images = sorted(os.listdir(low_res_folder))
#         self.high_res_images = sorted(os.listdir(high_res_folder))
#         self.transform = transform

#     def __len__(self):
#         return len(self.low_res_images)

#     def __getitem__(self, index):
#         low_res_image = Image.open(os.path.join(low_res_folder, self.low_res_images[index]))
#         high_res_image = Image.open(os.path.join(high_res_folder, self.high_res_images[index]))

#         if self.transform is not None:
#             low_res_image = self.transform(low_res_image)
#             high_res_image = self.transform(high_res_image)

#         return low_res_image, high_res_image

# # transform to tensor
# base_transform = transforms.Compose([
#     transforms.ToTensor()
# ])

# # original dataset
# dataset = CustomDataset(low_res_folder, high_res_folder, transform=base_transform)



# # === Splitting into train and val sets ===

# train_size = 0.8  # Proportion of data to be used for training
# dataset_size = len(dataset)
# split = int(train_size * dataset_size)
# train_indices = list(range(split))
# val_indices = list(range(split, dataset_size))

# # Create train dataset as a subset of the combined dataset
# train_dataset = Subset(dataset, train_indices)

# # Create val dataset as a subset of the combined dataset
# val_dataset = Subset(dataset, val_indices)



# # === Normalization ===
# normalize = Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])

# train_dataset = [(normalize(image), normalize(target)) for image, target in train_dataset]

# val_dataset = [(normalize(image), normalize(target)) for image, target in val_dataset]



# # === train_data augmentation ===

# # color jitter augmentation for training
# train_color_jitter = ColorJitter(brightness=0.4, contrast=0.4, saturation=0.4, hue=0.1)

# # augmentation factor
# augmentation_factor = 2

# # augmented datasets with random color jitter
# augmented_datasets = []
# for _ in range(augmentation_factor):
#     augmented_dataset = []
#     for image, target in train_dataset:
#         augmented_dataset.append((train_color_jitter(image), train_color_jitter(target)))
#     augmented_datasets.append(augmented_dataset)

# # combine original and augmented datasets
# combined_datasets = [train_dataset] + augmented_datasets
# combined_dataset = ConcatDataset(combined_datasets)



# # === final data loaders ===

# # Data loaders for train and val sets
# batch_size = 32
# train_loader = DataLoader(combined_dataset, batch_size=batch_size, shuffle=True)
# val_loader = DataLoader(val_dataset, batch_size=batch_size)

# # Number of samples in each set
# print(f"Number of training samples originally: {len(train_dataset)}, now augmented to: {len(combined_dataset)}")
# print(f"Number of val samples: {len(val_dataset)}")


KeyboardInterrupt: ignored

In [ ]:
low_res_folder = "/kaggle/working/dog_low_res"
high_res_folder = "/kaggle/input/animal-faces/afhq/train/dog"


# === creating dataset with all images ===
class CustomDataset(Dataset):
    def __init__(self, low_res_folder, high_res_folder, transform=None):
        self.low_res_folder = low_res_folder
        self.high_res_folder = high_res_folder
        self.low_res_images = sorted(os.listdir(low_res_folder))
        self.high_res_images = sorted(os.listdir(high_res_folder))
        self.transform = transform

    def __len__(self):
        return len(self.low_res_images)

    def __getitem__(self, index):
        low_res_image = Image.open(os.path.join(self.low_res_folder, self.low_res_images[index]))
        high_res_image = Image.open(os.path.join(self.high_res_folder, self.high_res_images[index]))

        if self.transform is not None:
            low_res_image = self.transform(low_res_image)
            high_res_image = self.transform(high_res_image)

        return low_res_image, high_res_image

# transform to tensor & normalize
normalize = Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])

base_transform = transforms.Compose([
    transforms.ToTensor(),
    normalize
])

# original dataset
dataset = CustomDataset(low_res_folder, high_res_folder, transform=base_transform)



# === Splitting into train and val sets ===

train_size = 0.8  # Proportion of data to be used for training
dataset_size = len(dataset)
split = int(train_size * dataset_size)
train_indices = list(range(split))
val_indices = list(range(split, dataset_size))

# Create train dataset as a subset of the combined dataset
train_dataset = Subset(dataset, train_indices)

# Create val dataset as a subset of the combined dataset
val_dataset = Subset(dataset, val_indices)



# === final data loaders ===

# Data loaders for train and val sets
batch_size = 128
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size)

# Number of samples in each set
print(f"Number of training samples originally: {len(train_dataset)}")
print(f"Number of val samples: {len(val_dataset)}")


### SRCNN

In [5]:
# SRCNN model
class SRCNN(nn.Module):
    def __init__(self):
        super(SRCNN, self).__init__()
        self.interpolation = nn.Upsample(scale_factor=4, mode='bicubic')
        self.conv1 = nn.Conv2d(3, 64, kernel_size=9, stride=1, padding=4)
        self.relu1 = nn.ReLU()
        self.conv2 = nn.Conv2d(64, 32, kernel_size=1, stride=1, padding=0)
        self.relu2 = nn.ReLU()
        self.conv3 = nn.Conv2d(32, 3, kernel_size=5, stride=1, padding=2)
        self.relu3 = nn.ReLU()

    def forward(self, x):
        x = self.interpolation(x)
        x = self.relu1(self.conv1(x))
        x = self.relu2(self.conv2(x))
        x = self.relu3(self.conv3(x))
        return x

# Training hardware
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# instance of the CNN model
model = SRCNN().to(device)

# hyperparameters
learning_rate = 0.001
num_epochs = 10

# loss function and optimizer
criterion = nn.MSELoss() # note: standard MSE is used, PSNR normally not used for training (just as metric at the end)
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Training process
for epoch in range(num_epochs):
    for input_data, desired_data in train_loader:
        # Move input and desired images to device
        input_data = input_data.to(device)
        desired_data = desired_data.to(device)

        # Forward pass
        output_images = model(input_data)

        # Calculate loss
        loss = criterion(output_images, desired_data)

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    # Print training loss per epoch
    psnr = 10 * math.log10(1 / loss.item())
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}, PSNR: {psnr}")

    # Validating the model (on validation data)
    for input_data, desired_data in val_loader:
        # Move input and desired images to device
        input_data = input_data.to(device)
        desired_data = desired_data.to(device)

        # Forward pass
        output_images = model(input_data)

        # Calculate loss
        loss = criterion(output_images, desired_data)

    # Print training loss per epoch
    psnr = 10 * math.log10(1 / loss.item())

    print(f"Loss (validation): {loss.item():.4f}, PSNR (validation): {psnr}")

# saving the model
torch.save(model.state_dict(), "SRCNN.pth")

Epoch [1/10], Loss: 0.1283, PSNR: 8.916304823903484
Loss (validation): 0.1736, PSNR (validation): 7.604106464489315
Epoch [2/10], Loss: 0.1220, PSNR: 9.137848423008013
Loss (validation): 0.1720, PSNR (validation): 7.645366720205501
Epoch [3/10], Loss: 0.0960, PSNR: 10.175107758089872
Loss (validation): 0.1720, PSNR (validation): 7.644895610697584
Epoch [4/10], Loss: 0.0786, PSNR: 11.04723886717254
Loss (validation): 0.1713, PSNR (validation): 7.663384318469921
Epoch [5/10], Loss: 0.0614, PSNR: 12.117327232179981
Loss (validation): 0.1710, PSNR (validation): 7.6689281909895435
Epoch [6/10], Loss: 0.1112, PSNR: 9.537440829689723
Loss (validation): 0.1712, PSNR (validation): 7.6646840091797594
Epoch [7/10], Loss: 0.1348, PSNR: 8.703944118333364
Loss (validation): 0.1722, PSNR (validation): 7.63884046002568
Epoch [8/10], Loss: 0.0904, PSNR: 10.438977789088515
Loss (validation): 0.1715, PSNR (validation): 7.657047145614541
Epoch [9/10], Loss: 0.0899, PSNR: 10.462438624621978
Loss (validatio

### FSRCNN

In [8]:
class FSRCNN(nn.Module):
    def __init__(self, d=56, s=12, m=4):
        super(FSRCNN, self).__init__()
        # Feature Extraction
        self.conv1 = nn.Conv2d(3, d, kernel_size=5, padding=2)
        self.relu1 = nn.PReLU(d)
        # Shrinking
        self.conv2 = nn.Conv2d(d, s, kernel_size=1)
        self.relu2 = nn.PReLU(s)
        # Non-linear Mapping
        self.mapping = nn.Sequential(*[nn.Sequential(
            nn.Conv2d(s, s, kernel_size=3, padding=1),
            nn.PReLU(s)
        ) for _ in range(m)])
        # Expanding
        self.conv3 = nn.Conv2d(s, d, kernel_size=1)
        self.relu3 = nn.PReLU(d)
        # Deconvolution
        self.deconv = nn.ConvTranspose2d(d, 3, kernel_size=9, stride=4, padding=4, output_padding=3)

    def forward(self, x):
        x = self.relu1(self.conv1(x))
        x = self.relu2(self.conv2(x))
        x = self.mapping(x)
        x = self.relu3(self.conv3(x))
        x = self.deconv(x)
        return x

# Training hardware
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# instance of the CNN model
model = FSRCNN().to(device)

# hyperparameters
learning_rate = 0.001
num_epochs = 40

# loss function and optimizer
criterion = nn.MSELoss() # note: standard MSE is used, PSNR normally not used for training (just as metric at the end)
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Training process
for epoch in range(num_epochs):
    for input_data, desired_data in train_loader:
        # Move input and desired images to device
        input_data = input_data.to(device)
        desired_data = desired_data.to(device)

        # Forward pass
        output_images = model(input_data)

        # Calculate loss
        loss = criterion(output_images, desired_data)

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    # Print training loss per epoch
    psnr = 10 * math.log10(1 / loss.item())
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}, PSNR: {psnr}")

    # Validating the model (on validation data)
    val_loss = 0
    number_batches = 0
    for input_data, desired_data in val_loader:
        number_batches += 1
        # Move input and desired images to device
        input_data = input_data.to(device)
        desired_data = desired_data.to(device)

        # Forward pass
        output_images = model(input_data)

        # Calculate loss
        loss = criterion(output_images, desired_data)
        val_loss += loss.item()
    
    val_loss_avg = val_loss / number_batches   

    # Print training loss per epoch
    psnr = 10 * math.log10(1 / val_loss_avg)

    print(f"Loss (validation): {val_loss_avg:.4f}, PSNR (validation): {psnr}")


# saving the model
torch.save(model.state_dict(), "FSRCNN.pth")


Epoch [1/40], Loss: 0.0323, PSNR: 14.905335504968562
Loss (validation): 0.0491, PSNR (validation): 13.092966777874882
Epoch [2/40], Loss: 0.0352, PSNR: 14.540321559611806
Loss (validation): 0.0446, PSNR (validation): 13.502632291955203
Epoch [3/40], Loss: 0.0283, PSNR: 15.480010865330476
Loss (validation): 0.0418, PSNR (validation): 13.783512723062803
Epoch [4/40], Loss: 0.0294, PSNR: 15.31094059723694
Loss (validation): 0.0383, PSNR (validation): 14.170864947352143
Epoch [5/40], Loss: 0.0312, PSNR: 15.065089931378733
Loss (validation): 0.0370, PSNR (validation): 14.323281188779323
Epoch [6/40], Loss: 0.0294, PSNR: 15.313538459650978
Loss (validation): 0.0346, PSNR (validation): 14.607680699022245
Epoch [7/40], Loss: 0.0150, PSNR: 18.234631743846535
Loss (validation): 0.0245, PSNR (validation): 16.10089254596251
Epoch [8/40], Loss: 0.0143, PSNR: 18.455967986404303
Loss (validation): 0.0232, PSNR (validation): 16.352925433769503
Epoch [9/40], Loss: 0.0205, PSNR: 16.891415753667722
Loss 

### Bicubic

In [10]:
class bicubic(nn.Module):
    def __init__(self):
        super(bicubic, self).__init__()
        self.interpolation = nn.Upsample(scale_factor=4, mode='bicubic')

    def forward(self, x):
        x = self.interpolation(x)
        return x

# Training hardware
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# instance of the CNN model
model = bicubic().to(device)

# loss function and optimizer
criterion = nn.MSELoss() # note: standard MSE is used, PSNR normally not used for training (just as metric at the end)

# Validating the model (on validation data)
for input_data, desired_data in val_loader:
    # Move input and desired images to device
    input_data = input_data.to(device)
    desired_data = desired_data.to(device)

    # Forward pass
    output_images = model(input_data)

    # Calculate loss
    loss = criterion(output_images, desired_data)

# Print training loss per epoch
psnr = 10 * math.log10(1 / loss.item())

print(f"Loss (validation): {loss.item():.4f}, PSNR (validation): {psnr}")


# saving the model
torch.save(model.state_dict(), "bicubic.pth")


Loss (validation): 0.0114, PSNR (validation): 19.41890646298794
